# Capítulo 1: Introdução ao Apache Arrow
## Curso: Apache Arrow + DuckDB

Neste capítulo você aprenderá:
- O que é Apache Arrow
- Integração com DuckDB
- Zero-copy operations
- Conversões Pandas ↔ Arrow
- Tipos de dados

In [ ]:
# Instalar dependências
# !pip install pyarrow duckdb pandas

In [ ]:
import pyarrow as pa
import duckdb
import pandas as pd
import time
from datetime import datetime, date

print(f"PyArrow version: {pa.__version__}")
print(f"DuckDB version: {duckdb.__version__}")

## 1.1 O que é Apache Arrow?

In [ ]:
# Criar Arrow table
arrow_table = pa.table({
    'id': [1, 2, 3],
    'name': ['Alice', 'Bob', 'Carol'],
    'age': [30, 25, 35]
})

print("Arrow Table criada:")
print(arrow_table)
print(f"\nTipo: {type(arrow_table)}")
print(f"Schema: {arrow_table.schema}")

## 1.2 Integração DuckDB + Arrow (Zero-Copy)

In [ ]:
# DuckDB pode ler/escrever Arrow diretamente
con = duckdb.connect()

# Query diretamente na Arrow table (ZERO-COPY!)
result = con.execute("SELECT * FROM arrow_table WHERE age > 25").arrow()

print("Resultado da query (zero-copy):")
print(result)
print(f"\nTipo: {type(result)}")

## 1.3 Conversões Pandas ↔ Arrow

In [ ]:
# Criar DataFrame Pandas
df = pd.DataFrame({
    'date': pd.date_range('2024-01-01', periods=5),
    'sales': [100, 150, 200, 175, 225],
    'region': ['North', 'South', 'North', 'East', 'South']
})

print("Pandas DataFrame:")
print(df)

# Converter para Arrow
arrow_from_pandas = pa.Table.from_pandas(df)
print("\nArrow Table:")
print(arrow_from_pandas)

In [ ]:
# Query com DuckDB
result_df = con.execute("""
    SELECT
        region,
        sum(sales) as total_sales,
        avg(sales) as avg_sales
    FROM arrow_from_pandas
    GROUP BY region
    ORDER BY total_sales DESC
""").df()

print("Resultado agregado:")
print(result_df)

## 1.4 Performance Comparison

In [ ]:
# Criar dados grandes
n = 1_000_000
data = {
    'id': range(n),
    'value': [i * 2.5 for i in range(n)],
    'category': ['A' if i % 3 == 0 else 'B' if i % 3 == 1 else 'C' for i in range(n)]
}

# Teste 1: Pandas
df_large = pd.DataFrame(data)
start = time.time()
result_pandas = con.execute("SELECT category, avg(value) FROM df_large GROUP BY category").df()
time_pandas = time.time() - start

# Teste 2: Arrow
arrow_large = pa.table(data)
start = time.time()
result_arrow = con.execute("SELECT category, avg(value) FROM arrow_large GROUP BY category").arrow()
time_arrow = time.time() - start

print(f"Tempo com Pandas: {time_pandas:.4f}s")
print(f"Tempo com Arrow:  {time_arrow:.4f}s")
print(f"Speedup: {time_pandas/time_arrow:.2f}x mais rápido com Arrow!")

## 1.5 Tipos de Dados Arrow

In [ ]:
# Criar tabela com tipos variados
complex_table = pa.table({
    'id': pa.array([1, 2, 3], type=pa.int32()),
    'name': pa.array(['Alice', 'Bob', 'Carol'], type=pa.string()),
    'balance': pa.array([1234.56, 7890.12, 3456.78], type=pa.decimal128(10, 2)),
    'birth_date': pa.array([date(1990, 1, 15), date(1985, 6, 20), date(1992, 3, 10)], type=pa.date32()),
    'is_active': pa.array([True, False, True], type=pa.bool_())
})

print("Arrow Table com tipos variados:")
print(complex_table)
print(f"\nSchema:\n{complex_table.schema}")

In [ ]:
# Query com tipos complexos
result_types = con.execute("""
    SELECT
        name,
        balance,
        is_active
    FROM complex_table
    WHERE is_active = true
""").df()

print("Resultado:")
print(result_types)

## Resumo do Capítulo 1

Você aprendeu:
- ✅ O que é Apache Arrow e por que é importante
- ✅ Integração nativa DuckDB + Arrow
- ✅ Zero-copy reads e vantagens de performance
- ✅ Como criar Arrow tables
- ✅ Conversão entre Pandas e Arrow
- ✅ Tipos de dados suportados